In [12]:
from __future__ import print_function
import pandas as pd
import shutil
import os
import sys
import tensorflow as tf

In [13]:
from tensorflow.keras.utils import Sequence

class PyDataset(Sequence):
    def __init__(self, data, labels, **kwargs):
        super().__init__(**kwargs)  # Call the parent class's __init__ method
        self.data = data
        self.labels = labels
        # Initialize any other attributes or perform additional setup

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [14]:
base_dir = 'dataset/data2' 

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
# )

train_datagen = ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    rotation_range=40,        # Randomly rotate images in the range (degrees, 0 to 40)
    width_shift_range=0.2,    # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,   # Randomly shift images vertically (fraction of total height)
    shear_range=0.2,          # Randomly shear images
    zoom_range=0.2,           # Randomly zoom in on images
    horizontal_flip=True,     # Randomly flip images horizontally
    fill_mode='nearest'       # Fill in newly created pixels with the nearest pixel value
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.4
)

In [16]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='validation'
)

Found 3829 images belonging to 59 classes.
Found 1509 images belonging to 59 classes.


In [17]:
def get_class_names(generator):
    return list(generator.class_indices.keys())

class_names = get_class_names(train_generator)
print("Class names:", class_names)

Class names: ['Apple', 'Banana', 'Bread', 'Cheese', 'Egg', 'Eggplant', 'Gandum', 'Honey', 'Mango', 'Milk', 'Mung beans', 'Mustard leaf', 'Noodles', 'Orange', 'Peanut Butter', 'Pineapple', 'Raw Squid', 'Red beans', 'Rice Flour', 'Shrimp', 'Soybeans raw', 'Spaghetti', 'Spinach', 'Sweet Corn', 'Sweet potato', 'Tapioca Flour', 'Tempe Mentah', 'Tomato', 'Wheat Flour', 'Yogurt', 'broccoli', 'carrot', 'coriander seeds', 'corn', 'cucumber', 'cumin seed', 'galangal', 'garlic', 'ginger', 'grated coconut', 'green chilli', 'green paprika', 'olive oil', 'peanuts', 'pepper powder', 'potato', 'raw chicken', 'raw cow meat', 'raw fish', 'raw goat meat', 'red chilli', 'red onion', 'red paprika', 'rice', 'salt', 'sugar', 'sun kara', 'tofu', 'turmeric']


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(59, activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 15, 15, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 59)             │        30,267 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,559,419 (51.73 MB)

 Trainable params: 13,559,419 (51.73 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',  # Memantau val_accuracy
    patience=5,  # Menghentikan pelatihan jika val_accuracy tidak meningkat dalam 5 epochs berturut-turut
    restore_best_weights=True,  # Mengembalikan bobot terbaik yang telah disimpan selama pelatihan
    verbose=1  # Menampilkan pesan ketika pelatihan dihentikan
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping_callback]
)

In [ ]:
saved_model_path = "./model_v1.h5"

model.save(saved_model_path)